In [2]:
!pip install sklearn
!pip install pandas

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     ---------------------------------------- 6.6/6.6 MB 5.4 MB/s eta 0:00:00
     -------------------------------------- 298.0/298.0 KB 6.1 MB/s eta 0:00:00
  Created wheel for sklearn: filename=sklearn-0.0-py2.py3-none-any.whl size=1309 sha256=11ac195f9432b79b0ed3eaf7251c28164fc48b87d11687070c1abbff7c511c0c
  Stored in directory: c:\users\madhav\appdata\local\pip\cache\wheels\22\0b\40\fd3f795caaa1fb4c6cb738bc1f56100be1e57da95849bfc897
Successfully built sklearn


You should consider upgrading via the 'c:\users\madhav\appdata\local\programs\python\python38-32\python.exe -m pip install --upgrade pip' command.


In [3]:
import pandas as pd
import sklearn
from scipy.io import arff

In [4]:
# load data set
data = arff.loadarff('TimeBasedFeatures-Dataset-30s-VPN.arff')
df = pd.DataFrame(data[0])
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14651 entries, 0 to 14650
Data columns (total 24 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   duration            14651 non-null  float64
 1   total_fiat          14651 non-null  float64
 2   total_biat          14651 non-null  float64
 3   min_fiat            14651 non-null  float64
 4   min_biat            14651 non-null  float64
 5   max_fiat            14651 non-null  float64
 6   max_biat            14651 non-null  float64
 7   mean_fiat           14651 non-null  float64
 8   mean_biat           14651 non-null  float64
 9   flowPktsPerSecond   14651 non-null  float64
 10  flowBytesPerSecond  14651 non-null  float64
 11  min_flowiat         14651 non-null  float64
 12  max_flowiat         14651 non-null  float64
 13  mean_flowiat        14651 non-null  float64
 14  std_flowiat         14651 non-null  float64
 15  min_active          14651 non-null  float64
 16  mean

In [ ]:
training_data = df.sample(frac=0.8, random_state=42)
testing_data = df.drop(training_data.index)

print(f"No. of training examples: {training_data.shape[0]}")
print(f"No. of testing examples: {testing_data.shape[0]}")

No. of training examples: 15006
No. of testing examples: 3752


In [ ]:
# removing class1 column
# 1. from traning set
training_out = [str(i) for i in training_data['class1']]
for i in range(len(training_out)):
  if(training_out[i] == "b'VPN'"):
    training_out[i] = 1
  else:
    training_out[i] = 0

trainX = training_data.drop(columns=['class1'])
trainX.head()

# 2. testing data
testing_out = [str(i) for i in testing_data['class1']]
for i in range(len(testing_out)):
  if(testing_out[i] == "b'VPN'"):
    testing_out[i] = 1
  else:
    testing_out[i] = 0

testX = testing_data.drop(columns=['class1'])
testX.head()

,duration,total_fiat,total_biat,min_fiat,min_biat,max_fiat,max_biat,mean_fiat,mean_biat,flowPktsPerSecond,...,mean_flowiat,std_flowiat,min_active,mean_active,max_active,std_active,min_idle,mean_idle,max_idle,std_idle
6,13978645.0,17.0,7.0,2249885.0,2064514.0,215483.375000,254157.181818,4.845279e+05,4.625930e+05,8.656061,...,116488.708333,3.342602e+05,1363299.0,2254176.2,3870047.0,1.010714e+06,1147658.0,1533121.80,2064514.0,4.218329e+05
9,8259704.0,18.0,9.0,2321059.0,2278426.0,269051.300000,330388.160000,6.395332e+05,6.372440e+05,6.900974,...,147494.714286,4.499576e+05,2368851.0,2687726.0,3162320.0,4.190281e+05,1702783.0,1980072.00,2270116.0,2.838815e+05
11,5545334.0,30.0,6.0,2349340.0,2315198.0,267851.550000,291859.684211,6.625533e+05,6.672003e+05,7.393603,...,138633.350000,4.681353e+05,2347008.0,2656002.0,2964996.0,4.369835e+05,1978303.0,2138028.50,2297754.0,2.258860e+05
13,12128582.0,21.0,8.0,8107194.0,7918721.0,461149.076923,527329.652174,1.616626e+06,1.669652e+06,4.204943,...,242571.640000,1.148809e+06,2762034.0,5957443.5,9152853.0,4.518991e+06,2066591.0,4992656.00,7918721.0,4.138081e+06
16,10300439.0,18.0,5.0,3029278.0,2997586.0,206406.551020,223922.586957,5.292343e+05,5.281801e+05,9.417074,...,107296.239583,3.749346e+05,1566070.0,2510109.5,3810504.0,1.123165e+06,1197148.0,1725752.75,2979791.0,8.396754e+05


## 1. Gaussian Naive Bayes


In [ ]:
from sklearn import metrics
from sklearn.naive_bayes import GaussianNB
gnb = GaussianNB()
gnb.fit(trainX, training_out)

GaussianNB()

In [ ]:
pred = gnb.predict(testX)
print("Gaussian Naive Bayes Accuracy: ",sklearn.metrics.accuracy_score(testing_out, pred)*100,"%")

Gaussian Naive Bayes Accuracy:  53.5181236673774 %


## 2. Decision Tree:
### Criterion: Entropy

In [ ]:
from sklearn import tree
clf = tree.DecisionTreeClassifier(criterion="entropy")
clf = clf.fit(trainX, training_out)

In [ ]:
pred = clf.predict(testX)
print("Accuracy: ",sklearn.metrics.accuracy_score(testing_out, pred)*100,"%")

Accuracy:  88.99253731343283 %


## 3. Random Forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(max_depth=200, random_state=42)
clf.fit(trainX, training_out)

RandomForestClassifier(max_depth=200, random_state=42)

In [ ]:
pred = clf.predict(testX)
print("Accuracy: ",sklearn.metrics.accuracy_score(testing_out, pred)*100,"%")

Accuracy:  91.4179104477612 %
